<a href="https://colab.research.google.com/github/OlegKuzmin1968/Oleg_Kuzmin/blob/main/Oleg_Kuzmin/%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2D_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%BE%D0%B9_%D0%9D%D0%A1_Conv2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#*Создание 2D модели сверточной НС Conv2D для распознования выпота. ЗАДАЧА*

Работаем с выборкой из 14 пациентов начиная с 303 и до 396 (только по тем для которых сделана сегментация преврального выпота).

Цель:

    1. Написать код по преобразованию DICOM файла в np-массив для каждого пациента.

    2. Предусмотреть возможность сохранения полученного np-массив с соответствующим идентификатором пациента на googl Disk

    3. Написать генератор данных, который использую функцию преобразования массива сможет подавать эти данные в НС
    
    ...для каждого пациента  из слайсов независимо от пациентов, при этом каждому слайсу присвоить двузначный правильный ответ: имеется ли на этом слайсе попаженная часть (выпот).x_train

# Загрузки

In [30]:
from nibabel.viewers import OrthoSlicer3D
from nibabel import nifti1
import nibabel as nib
from matplotlib import pylab as plt
import matplotlib
import numpy as np
from numpy.core.multiarray import dtype
from matplotlib import pyplot, cm
import os
from PIL import Image # Синтаксис и описание функции Image.open():
import random

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler # Функции для нормализации данных
from sklearn import preprocessing # Пакет предварительной обработки данных
from tensorflow.keras import utils #Используем дял to_categoricall

from tensorflow.keras.datasets import mnist # Из открытой библиотеки программ Tensorflow для Keras, раздел набор данных, импортируем базу ч/б картинок 28х28 mnist

**Пакет для создания и работы нейронной сети**

In [32]:
from tensorflow.keras.models import Sequential # Из открытой библиотеки программ Tensorflow для Keras, раздел models, программу создания пустой модели НС
from tensorflow.keras.layers import Dense,BatchNormalization, Dropout, Conv2D, Flatten, MaxPooling2D # ...  раздел layers (слои), программу создания слоев НС
from tensorflow.keras.optimizers import Adam,  Adadelta  # оптимизаторы (optimizers) Adam (0.001)


**Графический покет для прорисовки графиков и картинок**

In [ ]:
#!rm -R /content/drive/MyDrive/Baza/Stag_1/Effusions_Directory/LUNG1-420 # УДАЛЕНИЕ НЕПУСТОЙ ПАПКИ В COLABE

rm: cannot remove '/content/drive/MyDrive/Baza/Stag_1/Effusions_Directory/LUNG1-420': No such file or directory


In [ ]:
#!rm -R /content/drive/MyDrive/Baza/Stag_1/np

https://drive.google.com/drive/folders/1QHsJbmCAFGp6Hl-tgtHutvToTy2I1Lj_?usp=drive_link

**Дмитрий Черемисов** Сюда загрузил первый проект и датасет.

**Из NB "Формирование x_train и y_train на основе 2D массива", берем список test-овой выборки соответствующих пациентов.**

In [33]:
keys_list_134 = ['LUNG1-303', 'LUNG1-307', 'LUNG1-312', 'LUNG1-325', 'LUNG1-331', 'LUNG1-336', 'LUNG1-361', 'LUNG1-362', 'LUNG1-366', 'LUNG1-367', 'LUNG1-376', 'LUNG1-377', 'LUNG1-381', 'LUNG1-396']


In [34]:
len(keys_list_134)

14


# Работа с оригинальным изображением

### Загрузки библиотек для работы с DICOM

In [35]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.7 MB/s eta 0:00:00


In [36]:
# Единственный интересный момент здесь заключается в том, как я упоминал в разделе "Пакет pydicom",
# что пакет pydicom импортируется как dicom, поэтому будьте осторожны с этим.

import pydicom           # запустился так
import pydicom.data      # + metod

## **Путь к локальной папке оригинальных снимков для test**

In [37]:
# Присвоим переменной q путь к папке где хранятся все оригинальные снимки DICOM
q = '/content/drive/MyDrive/Baza/Stag_1/Original_Directory'
print (' q-', q,'\n','Тип переменной q -', type(q))

 q- /content/drive/MyDrive/Baza/Stag_1/Original_Directory 
 Тип переменной q - <class 'str'>


In [38]:
# Путь к оригинальным снимкам тестовой выборки на моем Googl Disk
Original_Directory = '/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'

Original_list = os.listdir(Original_Directory)   # занесем всех пациентов с оригинальными снимками в список
Original_list.sort()                             # отсортируем список
Original_list.pop(0)                             # удалим первый элемент

print('Сам список после сортировки: ', Original_list)
pac = len(Original_list)                         # количество пациентов с оригинальными снимками в тестовой выборке DS
print('Количество пациентов-', pac)


Сам список после сортировки:  ['LUNG1-381', 'LUNG1-396']
Количество пациентов- 2


## Функция составления списка слайсов

In [39]:
def LUNG_n_slise (n, Original_list):

  ''' Функция составления списка слайсов оригинального снимка
      Вход:  n - порядковый номер пациента в списке
             Original_list - список пациентов с оригинальными списками
      Выход: Список слайсов в оригинальном снимке одного пациента '''

  lun_1 = Original_list [n]
  #print(Original_list [n],'n')
  print('   Файл со снимком одного из пациентов',lun_1)
  Original_list2 = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/' +lun_1)
  lun_2 = Original_list2 [0]
  #print('lun_2',lun_2)
  Original_list3 = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/'+ lun_1 + '/' + lun_2)
  lun_3 = Original_list3 [0]
  #print('lun_3',lun_3)
  base = '/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/'+ lun_1 + '/' + lun_2 + '/' + lun_3
  Original_list4 = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/'+ lun_1 + '/' + lun_2 + '/' + lun_3)
  Original_list4.sort()
  #k = str('lung_' + str(n))
  #print(len(Original_list4))
  #print('Original_list4', Original_list4 [12])
  return Original_list4, base


In [ ]:
LUNG_n_slise (0,Original_list)

   Файл со снимком одного из пациентов LUNG1_396


(['1-001.dcm',
  '1-002.dcm',
  '1-003.dcm',
  '1-004.dcm',
  '1-005.dcm',
  '1-006.dcm',
  '1-007.dcm',
  '1-008.dcm',
  '1-009.dcm',
  '1-010.dcm',
  '1-011.dcm',
  '1-012.dcm',
  '1-013.dcm',
  '1-014.dcm',
  '1-015.dcm',
  '1-016.dcm',
  '1-017.dcm',
  '1-018.dcm',
  '1-019.dcm',
  '1-020.dcm',
  '1-021.dcm',
  '1-022.dcm',
  '1-023.dcm',
  '1-024.dcm',
  '1-025.dcm',
  '1-026.dcm',
  '1-027.dcm',
  '1-028.dcm',
  '1-029.dcm',
  '1-030.dcm',
  '1-031.dcm',
  '1-032.dcm',
  '1-033.dcm',
  '1-034.dcm',
  '1-035.dcm',
  '1-036.dcm',
  '1-037.dcm',
  '1-038.dcm',
  '1-039.dcm',
  '1-040.dcm',
  '1-041.dcm',
  '1-042.dcm',
  '1-043.dcm',
  '1-044.dcm',
  '1-045.dcm',
  '1-046.dcm',
  '1-047.dcm',
  '1-048.dcm',
  '1-049.dcm',
  '1-050.dcm',
  '1-051.dcm',
  '1-052.dcm',
  '1-053.dcm',
  '1-054.dcm',
  '1-055.dcm',
  '1-056.dcm',
  '1-057.dcm',
  '1-058.dcm',
  '1-059.dcm',
  '1-060.dcm',
  '1-061.dcm',
  '1-062.dcm',
  '1-063.dcm',
  '1-064.dcm',
  '1-065.dcm',
  '1-066.dcm',
  '1-067.d

## Создание np-файлов для 2D модели test


### Функция создания словаря путей к слайсам тестовой выборки

In [40]:
counter = 0             # сквозной счетчик слайсов тестовой выборки
orig_dist = {}          # словарь- сквозной порядковый номер слайса:путь к слайсу конкретного пациента

for pa in range (pac):  # цикл обращения ко всем папкам пациентов

  pass_dicom_list, base = LUNG_n_slise (pa,Original_list)              # список слайсов в оригинальном снимке пациента, путь к папке пациента
  #print('Количество слайсов в файле пациента:', len(pass_dicom_list))
  #print('Название очередного слайса из списка:', pass_dicom_list[pa])
  #print('Путь к слайсам конкретного пациента -', base)

  for slis in range (len(pass_dicom_list)):  # цикл работы со слайсом очередного пациента
    x_pa = base + '/' + pass_dicom_list[slis]    # собираем полный путь к файлу-слайсу очередного пациента
    print('x_pa',x_pa)
    print('Сквозной порядковый номер слайса -', counter)

    orig_dist[counter] = x_pa                    # добавляем в словарь новую пару
    counter += 1                                 # изменяем значение счетчика на 1

print('все, словарь для x_train готов')



   Файл со снимком одного из пациентов LUNG1-381
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-001.dcm
Сквозной порядковый номер слайса - 0
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-002.dcm
Сквозной порядковый номер слайса - 1
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-003.dcm
Сквозной порядковый номер слайса - 2
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-004.dcm
Сквозной порядковый номер слайса - 3
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-005.dcm
Сквозной порядковый номер слайса - 4
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-006.dcm
Сквозной порядко

In [ ]:
orig_dist[164] # смотрим содержание словаря

'/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-396/05-01-2010-NA-NA-28227/0.000000-NA-05554/1-031.dcm'

### Функция y_gen генератор


In [41]:
# Чтение массива y_train из googl diska в настоящий NB_colab

with open('/content/drive/MyDrive/Baza/Stag_1/Effusions_np/y_train_1876.txt', 'r') as f:
  y_train_1876 = np.loadtxt(f)
  print('y_train_1876', y_train_1876, type(y_train_1876), y_train_1876.shape)

y_train_1876 [0. 0. 0. ... 0. 0. 0.] <class 'numpy.ndarray'> (1876,)


In [93]:
a = np.array ([1, 2, 3, 4], dtype=np.float64)
print(a)
#array([ 1.,  2.,  3.,  4.])
y_train = y_train_1876.astype(np.int16)
print(y_train)
#array([1, 2, 3, 4])

[1. 2. 3. 4.]
[0 0 0 ... 0 0 0]


In [94]:
y_train [:200]  # вся последовательность представляет собой, с определенной долей приближения, гармоничный ряд.

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int16)

In [97]:
def batch_y (y_train):            # на вход функции подаем список слайсов обучающей выборки  с параметром sufle
  for i in range(len(y_train)):   # длина цикла определяется размером количеством слайсов обучающей выборки
    print ('i', i)
    print ('y_train [i]',y_train [i])
    yield y_train [i]             # при очередном обращении выдает соответствующее значение y_train

# На входе у нас есть список y_train_1876 содержащий правильные ответы для каждого слайса из test выборки

y_gen = batch_y (y_train)    # функцию: <class 'function'> batch_y - превращаем в генератор y_gen <generator object batch_it at 0x7f3ea189f990>

In [98]:
print('1-е обращение', next (y_gen))
print('2-е обращение', next (y_gen))
print('3-е обращение', next (y_gen))

i 0
y_train [i] 0
1-е обращение 0
i 1
y_train [i] 0
2-е обращение 0
i 2
y_train [i] 0
3-е обращение 0


In [99]:
for k in range (30):
  print(k,'-е','обращение', next (y_gen))

i 3
y_train [i] 0
0 -е обращение 0
i 4
y_train [i] 0
1 -е обращение 0
i 5
y_train [i] 0
2 -е обращение 0
i 6
y_train [i] 0
3 -е обращение 0
i 7
y_train [i] 0
4 -е обращение 0
i 8
y_train [i] 0
5 -е обращение 0
i 9
y_train [i] 0
6 -е обращение 0
i 10
y_train [i] 0
7 -е обращение 0
i 11
y_train [i] 0
8 -е обращение 0
i 12
y_train [i] 0
9 -е обращение 0
i 13
y_train [i] 0
10 -е обращение 0
i 14
y_train [i] 0
11 -е обращение 0
i 15
y_train [i] 0
12 -е обращение 0
i 16
y_train [i] 0
13 -е обращение 0
i 17
y_train [i] 0
14 -е обращение 0
i 18
y_train [i] 0
15 -е обращение 0
i 19
y_train [i] 0
16 -е обращение 0
i 20
y_train [i] 0
17 -е обращение 0
i 21
y_train [i] 0
18 -е обращение 0
i 22
y_train [i] 0
19 -е обращение 0
i 23
y_train [i] 0
20 -е обращение 0
i 24
y_train [i] 1
21 -е обращение 1
i 25
y_train [i] 1
22 -е обращение 1
i 26
y_train [i] 1
23 -е обращение 1
i 27
y_train [i] 1
24 -е обращение 1
i 28
y_train [i] 1
25 -е обращение 1
i 29
y_train [i] 1
26 -е обращение 1
i 30
y_train [i] 1

In [100]:
next (y_gen)

i 33
y_train [i] 1


1

### Функция x_gen через словарь

In [101]:
len_dist = list(orig_dist.keys())   # все ключи словаря перевели в список ключей
# print (len_dist[90:100])
# len(len_dist)                     # теперь можем посмотреть размер словаря (через дляну списка)

def batch_x (orig_dist): # на вход функции подаем список слайсов обучающей выборки  с параметром sufle
  '''
     вход: orig_dist- словарь: сквозной порядковый номер слайса:путь к слайсу конкретного пациента
     выход: np_orig- слайс DICOM преобразованый в np массив размерностью (512,512,1)

  '''
  for slis in range (len(len_dist)-1):  # цикл работы со слайсами ограничен длиной словаря -1
    q = orig_dist[slis]
    print('q', q)
    ds = pydicom.dcmread(q)                              # Функция считывает одно изображение и возвращает объект pydicom.dataset.FileDataset
    test_str = ds.PatientID                              # читаем из файла ID пациента
    print('Patient', test_str, 'Slis', slis)
    np_orig = np.array(ds.pixel_array, dtype= 'int16')   # можно изменить размер переменной

    q_np = np.reshape (np_orig,[512,512,1])              # Добавим размерность для подачи в Conv2D слой НС (принимает только 3-х мерный массив)
    print('q_np.shape', q_np.shape)
    print('все')

    yield q_np


In [102]:
x_gen = batch_x (orig_dist)   # функцию: <class 'function'> batch_x - превращаем в генератор x_gen <generator object batch_it at 0x7f3ea189f990>

In [103]:
print('1-е обращение', next (x_gen))
print('2-е обращение', next (x_gen))
print('3-е обращение', next (x_gen))

q /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-001.dcm
Patient LUNG1-381 Slis 0
q_np.shape (512, 512, 1)
все
1-е обращение [[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]]
q /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-002.dcm
Patient LUNG1-381 Slis 1
q_np.shape (512, 512, 1)
все
2-е обращение [[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]]
q /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-201

## НС Conv2D

In [15]:

from tensorflow.keras.models import Sequential #Сеть прямого распространения
#Базовые слои для счёрточных сетей
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator # работа с изображениями
from tensorflow.keras.optimizers import Adam, Adadelta # оптимизаторы
from tensorflow.keras import utils #Используем дял to_categoricall
from tensorflow.keras.preprocessing import image #Для отрисовки изображений
from google.colab import files #Для загрузки своей картинки
import numpy as np #Библиотека работы с массивами
import matplotlib.pyplot as plt #Для отрисовки графиков
from PIL import Image #Для отрисовки изображений
import random #Для генерации случайных чисел
import math # Для округления
import os #Для работы с файлами
# подключем диск
from google.colab import drive

%matplotlib inline

In [12]:
from tensorflow.keras import activations

In [104]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu' ))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))



In [105]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [76]:
batch_size = 5
# Размер мини-выборки

# Количество изображений для обучения
nb_train_samples = 175
# Количество изображений для проверки
nb_validation_samples = 25
# Количество изображений для тестирования
nb_test_samples = 37

In [107]:
model.fit_generator(x_gen, y_gen, epochs=3, verbose=1) #, steps_per_epoch=nb_train_samples // batch_size )


q /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-005.dcm
Patient LUNG1-381 Slis 4
q_np.shape (512, 512, 1)
все


<ipython-input-107-d7c65ae4603e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_gen, y_gen, epochs=30, verbose=1) #, steps_per_epoch=nb_train_samples // batch_size )


TypeError: ignored